In [2]:
import pandas as pd

In [3]:
df15 = pd.read_csv('dataset/2015.csv', encoding='cp1252', sep=';')
df16 = pd.read_csv('dataset/2016.csv', encoding='cp1252', sep=';')
df17 = pd.read_csv('dataset/2017.csv', encoding='cp1252', sep=';')
df18 = pd.read_csv('dataset/2018.csv', encoding='cp1252', sep=';')
df19 = pd.read_csv('dataset/2019.csv', encoding='cp1252', sep=';')
df20 = pd.read_csv('dataset/2020.csv', encoding='cp1252', sep=';')
df21 = pd.read_csv('dataset/2021.csv', encoding='cp1252', sep=';')
df22 = pd.read_csv('dataset/2022.csv', encoding='cp1252', sep=';')
df23 = pd.read_csv('dataset/2023.csv', encoding='cp1252', sep=';')
df24 = pd.read_csv('dataset/2024.csv', encoding='cp1252', sep=';')

In [4]:
df = pd.concat([df15, df16, df17, df18, df19, df20, df21, df22, df23, df24])

In [6]:
desastres = list(df['COBRADE'].unique())

In [7]:
id_full_disasters = {}
for d in desastres:
    cod = d[:5]
    id_full_disasters[cod] = d[8:]

In [8]:
keep_cols = [
    'UF', 'Municï¿½pio', 'Registro', 'COBRADE',
    'Status', 'Populaï¿½ï¿½o', 'DH_Mortos', 'DH_Feridos', 'DH_Enfermos',
    'DH_Desabrigados', 'DH_Desalojados'
]
df = df[keep_cols]
df['DH_Feridos_Enfermos'] = df['DH_Feridos'] + df['DH_Enfermos']
df['DH_Desabrigados_Desalojados'] = df['DH_Desabrigados'] + df['DH_Desalojados']
df = df.drop(['DH_Feridos', 'DH_Enfermos', 'DH_Desabrigados', 'DH_Desalojados'], axis=1)

In [9]:
df = df.rename(columns={'Municï¿½pio': 'Municipio', 'COBRADE': 'Desastre', 'Populaï¿½ï¿½o': 'Populacao'})
df['Desastre'] = df['Desastre'].str[:5]
df['Data'] = pd.to_datetime(df['Registro'], format='%d/%m/%Y')
df = df.drop(['Registro'], axis=1)
df = df.reset_index(drop=True)

In [129]:
df

,UF,Desastre,DH_Mortos,DH_Feridos_Enfermos,DH_Desabrigados_Desalojados,Data
0,RJ,13215,0.0,0.0,0.0,2016-01-05
1,RS,12100,0.0,0.0,280.0,2015-12-30
2,RS,12100,0.0,0.0,0.0,2015-12-30
3,SC,12200,0.0,0.0,0.0,2016-01-05
4,RS,13214,0.0,0.0,0.0,2015-12-30
...,...,...,...,...,...,...
64057,PR,15110,0.0,254.0,0.0,2024-04-05
64058,CE,13214,1.0,0.0,0.0,2024-01-10
64059,BA,14110,0.0,0.0,0.0,2024-01-18
64060,RN,11410,0.0,0.0,0.0,2024-01-01


## 2. Computation of Frequent Eventsets:

- Remove "itemsets" with "Desastre" type count less than min_sup

In [10]:
min_sup = 0.5/100
count = dict(df['Desastre'].value_counts())
tot = sum(count.values())
disasters_to_drop = set()

for d, c in count.items():
    if c/tot < min_sup:
        disasters_to_drop.add(d)

# doencas virais
disasters_to_drop.add('15110')

df_min_sup = df[~df['Desastre'].isin(disasters_to_drop)]

## 3. Calculating frequent sequences

1. With pycspade
2. Transform into:

```
Your data needs to be in a particular format similar to the following:

1 1 3 8 37 42
1 2 4 4 11 37 42
2 1 2 10 73
2 2 1 72
2 3 3 4 24 77
...
The first number is the sequence index, the second is the event index, the third is the number of elements, followed by the element, space separated

data = [
    [1, 10, [3, 4]],
    [1, 15, [1, 2, 3]],
    [1, 20, [1, 2, 6]],
    [1, 25, [1, 3, 4, 6]],
    [2, 15, [1, 2, 6]],
    [2, 20, [5]],
    [3, 10, [1, 2, 6]],
    [4, 10, [4, 7, 8]],
    [4, 20, [2, 6]],
    [4, 25, [1, 7, 8]]
]

To construct this:
1. Identify all unique items: Extract all unique values from your Desastre column.
2. Populate id-lists: For each unique Desastre item, create a list of (sid, eid) pairs where that item appeared.
Here’s an illustrative partial example of your data in vertical format:

```

In [11]:
d_ids = {}
for id, d in enumerate(df_min_sup['Desastre'].unique()):
    d_ids[d] = id + 1

dates_ids = {}
for id, d in enumerate(df_min_sup.sort_values('Data')['Data'].unique()):
    dates_ids[str(d.astype('datetime64[D]'))] = id + 1

df_min_sup['UF_Municipio'] = df_min_sup['UF'].astype(str) + '-' + df_min_sup['Municipio']
uf_mun_ids = {}
for id, uf_mun in enumerate(df_min_sup['UF_Municipio'].unique()):
    uf_mun_ids[uf_mun] = id + 1

/tmp/ipykernel_356724/653276908.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_sup['UF_Municipio'] = df_min_sup['UF'].astype(str) + '-' + df_min_sup['Municipio']


In [12]:
vertical_dataset = {}
for uf_mun in df_min_sup['UF_Municipio'].unique():
    events = pd.DataFrame(df_min_sup[df_min_sup['UF_Municipio'] == uf_mun])
    events['Data'] = events['Data'].astype('datetime64[D]')
    items = events[['Data', 'Desastre']]
    vertical_dataset[uf_mun] = items

In [13]:
max_id = 0

spade_input = []
for uf_city, events in vertical_dataset.items():
    entries = []
    for date in events.sort_values('Data')['Data'].unique():
        disasters = list(set(events[events['Data'] == date]['Desastre'].values))
        np_date = str(pd.Timestamp(date).to_numpy().astype('datetime64[D]'))
        entries.append([uf_mun_ids[uf_city], dates_ids[np_date], disasters])
        # entries.append([uf_mun_ids[uf_city], dates_ids[np_date], list(map(lambda x: d_ids[x], disasters))])
    spade_input += entries

In [160]:
spade_input

[[1, 48, ['12300']],
 [1, 285, ['13214', '13215']],
 [1, 295, ['13214']],
 [1, 548, ['13214']],
 [1, 573, ['13214']],
 [1, 917, ['13214']],
 [1, 965, ['13214']],
 [1, 1167, ['13214']],
 [1, 1490, ['13213']],
 [1, 1545, ['13214']],
 [1, 1557, ['11321']],
 [1, 2155, ['13214']],
 [1, 2239, ['12300']],
 [1, 2571, ['12300']],
 [1, 2830, ['13214']],
 [1, 2900, ['13214']],
 [1, 3089, ['13215']],
 [2, 213, ['12300']],
 [2, 214, ['12200']],
 [2, 275, ['12200']],
 [2, 280, ['12100']],
 [2, 718, ['12100']],
 [2, 721, ['12200']],
 [2, 828, ['13215']],
 [2, 1210, ['12200']],
 [2, 1539, ['14110']],
 [2, 1789, ['14110']],
 [2, 2155, ['14110']],
 [2, 2472, ['14110']],
 [2, 2750, ['12100']],
 [2, 2933, ['13214']],
 [3, 223, ['13213']],
 [3, 280, ['12100']],
 [3, 1520, ['14110']],
 [3, 2153, ['14110']],
 [3, 2483, ['14110']],
 [3, 2807, ['13214']],
 [3, 2838, ['13214']],
 [3, 2933, ['13214']],
 [4, 18, ['12200']],
 [4, 239, ['13214']],
 [4, 285, ['12200']],
 [4, 307, ['13214']],
 [4, 309, ['12200']],
 [

In [14]:
from pycspade.helpers import spade, print_result

In [15]:
result = spade(data=spade_input, support=0.15)
print_result(result)

fmt_rules = []

for rule in result['mined_objects']:
    fmt_rules.append(list(map(lambda x: id_full_disasters[str(x.__dict__['elements'][0])], rule.__dict__['items'])))

fmt_rules

   Occurs     Accum   Support    Confid      Lift                                                                         Sequence
      861       861 0.1808444       N/A       N/A                                                                          (12100) 
     1109      1109 0.2329343       N/A       N/A                                                                          (12200) 
      784       784 0.1646713 0.7069432 1.2042063                                                                 (12200)->(13214) 
      794       794 0.1667717       N/A       N/A                                                                          (12300) 
     2795     11164 0.5870615       N/A       N/A                                                                          (13214) 
     1690      3518 0.3549674 0.6046512 1.0299621                                                                 (13214)->(13214) 
     1012      1012 0.2125604 0.5988166 1.0200235                            

[['Inundaï¿½ï¿½es'],
 ['Enxurradas'],
 ['Enxurradas', 'Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Alagamentos'],
 ['Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Tempestade Local/Convectiva - Chuvas Intensas',
  'Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Tempestade Local/Convectiva - Chuvas Intensas',
  'Tempestade Local/Convectiva - Chuvas Intensas',
  'Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Tempestade Local/Convectiva - Chuvas Intensas', 'Estiagem'],
 ['Tempestade Local/Convectiva - Vendaval'],
 ['Tempestade Local/Convectiva - Vendaval',
  'Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Estiagem'],
 ['Estiagem', 'Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Estiagem',
  'Tempestade Local/Convectiva - Chuvas Intensas',
  'Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Estiagem', 'Estiagem'],
 ['Estiagem', 'Estiagem', 'Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Estiagem', 'Estiagem', 'Estiagem'],
 ['Estiagem',
  'Estiagem'

In [17]:
result = spade(data=spade_input, support=0.15)
print_result(result)

fmt_rules = []

for rule in result['mined_objects']:
    fmt_rules.append(list(map(lambda x: str(x.__dict__['elements'][0]), rule.__dict__['items'])))

fmt_rules

   Occurs     Accum   Support    Confid      Lift                                                                         Sequence
      861       861 0.1808444       N/A       N/A                                                                          (12100) 
     1109      1109 0.2329343       N/A       N/A                                                                          (12200) 
      784       784 0.1646713 0.7069432 1.2042063                                                                 (12200)->(13214) 
      794       794 0.1667717       N/A       N/A                                                                          (12300) 
     2795     11164 0.5870615       N/A       N/A                                                                          (13214) 
     1690      3518 0.3549674 0.6046512 1.0299621                                                                 (13214)->(13214) 
     1012      1012 0.2125604 0.5988166 1.0200235                            

[['12100'],
 ['12200'],
 ['12200', '13214'],
 ['12300'],
 ['13214'],
 ['13214', '13214'],
 ['13214', '13214', '13214'],
 ['13214', '14110'],
 ['13215'],
 ['13215', '13214'],
 ['14110'],
 ['14110', '13214'],
 ['14110', '13214', '13214'],
 ['14110', '14110'],
 ['14110', '14110', '13214'],
 ['14110', '14110', '14110'],
 ['14110', '14110', '14110', '13214'],
 ['14110', '14110', '14110', '14110'],
 ['14110', '14110', '14110', '14110', '14110'],
 ['14110', '14110', '14110', '14110', '14110', '14110'],
 ['14110', '14110', '14110', '14110', '14110', '14110', '14110'],
 ['14110', '14120'],
 ['14120'],
 ['14120', '14110'],
 ['14120', '14120'],
 ['14132']]

## 4. Subgroup Discovery

In [19]:
import pandas as pd
from collections import defaultdict

# Load disaster events
df = df_min_sup

# Group disasters per location, ordered by date
location_sequences = df.sort_values("Data").groupby("UF_Municipio")["Desastre"].apply(list)

# Convert sequences from SPADE into string form for easier matching
frequent_sequences = fmt_rules  # sample
sequence_names = [f"Seq_{i}" for i in range(len(frequent_sequences))]

# Binary table of sequence presence per location
binary_table = defaultdict(dict)
for loc, seq in location_sequences.items():
    for idx, fseq in enumerate(frequent_sequences):
        name = sequence_names[idx]
        # Check if the frequent sequence is a subsequence of this location's disaster list
        def is_subsequence(small, big):
            # import pdb; pdb.set_trace()
            it = iter(big)
            ok = all(x in it for x in small)
            return ok

        binary_table[loc][name] = int(is_subsequence(fseq, seq))

# Convert to DataFrame
sequence_df = pd.DataFrame.from_dict(binary_table, orient='index').reset_index().rename(columns={'index': 'UF_Municipio'})

# Merge with original static info (UF, Municipio, Populacao)
static_df = df.drop_duplicates("UF_Municipio")[['UF_Municipio', 'UF', 'Municipio', 'Populacao']]
final_df = pd.merge(static_df, sequence_df, on="UF_Municipio")

okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkkk
okkk

In [20]:
final_df

,UF_Municipio,UF,Municipio,Populacao,Seq_0,Seq_1,Seq_2,Seq_3,Seq_4,Seq_5,...,Seq_16,Seq_17,Seq_18,Seq_19,Seq_20,Seq_21,Seq_22,Seq_23,Seq_24,Seq_25
0,RJ-Duque de Caxias,RJ,Duque de Caxias,855046,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,RS-Manoel Viana,RS,Manoel Viana,7074,1,1,1,1,1,0,...,1,1,0,0,0,0,0,0,0,0
2,RS-Paraï¿½so do Sul,RS,Paraï¿½so do Sul,7336,1,0,0,0,1,1,...,1,0,0,0,0,0,0,0,0,0
3,SC-ï¿½guas Mornas,SC,ï¿½guas Mornas,5546,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,RS-Sï¿½o Vicente do Sul,RS,Sï¿½o Vicente do Sul,8440,0,1,1,0,1,1,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4756,PR-Matinhos,PR,Matinhos,29426,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4757,SP-Embu-Guaï¿½u,SP,Embu-Guaï¿½u,62846,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4758,MG-Sï¿½o Vicente de Minas,MG,Sï¿½o Vicente de Minas,7008,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4759,MG-Olï¿½mpio Noronha,MG,Olï¿½mpio Noronha,2533,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
import pysubgroup as ps

# Suppose we are analyzing 'Seq_2'
target = ps.BinaryTarget("Seq_2", 1)
search_space = ps.create_selectors(final_df, ['UF'])  # Could add Populacao bins, Municipio, etc.
qf = ps.WRAccQF()

result = ps.BeamSearch(beam_width=5, depth=2).execute(
    final_df, target, search_space, result_set_size=10, quality_function=qf
)

# Inspect results
for rule in result:
    print(rule.subgroup_description, rule.quality, rule.statistics)


TypeError: '<=' not supported between instances of 'int' and 'list'